This is a notebook that I will use to define the functions to automate scraping from Nepremicnine.net. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import pandas as pd
import numpy as np
import bs4
import lxml.etree as xml
import time
import datetime
from tqdm import tqdm
import random

Define path to the webdriver and define webdriver as driver. 

In [3]:
PATH = '/Users/Asus/Downloads/chromedriver' # Defines where my cromedriver package is installed. Opens an automated Chrome window.
driver = webdriver.Chrome(PATH) # names webdriver as driver and assigns path. 
driver.quit() # closes the driver so the window is not always on. 

In [179]:
# give base url to the site I want to scrape.
url = 'https://www.nepremicnine.net/oglasi-prodaja/'

In [181]:
driver = webdriver.Chrome(PATH) 
driver.get(url) # automatically opens Nepremicnine.net
# driver.quit() # this closes the window. 

In [182]:
html = driver.page_source
webPage = bs4.BeautifulSoup(html, "lxml")

In [110]:
#print(webPage.prettify())
# This is how the full HTML code looks like.
# Probably not needed, since we can select exactly what we want to look at in Chrome, but its there. 

We need to find what regions there are listed as options and how many pages of listings there are for each region. 
At this moment, I am only considering option of 'Selling', 'Renting' will come later. 

In [183]:
# find the number of total pages. 
# print(webPage.find('div', id= 'pagination').find('span').text)
tpages = int(webPage.find('div', id= 'pagination').find('span').text)
tpages

618

In [ ]:
# filtering out a year as an int, provided that 'atribut leto' is not None
for index, ad in enumerate(webPage.find_all('div', itemprop='item')):
    print({index}, (int(ad.find('a', class_='ikona-sh3 utility')['data-id'])))
    year_raw = ad.find('span', class_='atribut leto')
    year = int(year_raw.text.split(':')[-1]) if year_raw != None else print(np.nan)
    print(year)
    print('------------')

In [10]:
# Starting DataFrame.
d = []
colNames = ['AdID','Title','Type', 'YearBuilt', 'Link', 'Seller', 'Size','SalePrice', 'lastUpdate']
data1 = pd.DataFrame(d, columns= colNames)
data1.shape

(0, 9)

In [74]:
del data

In [184]:
data = data.iloc[0:0]
data

,AdID,Title,Type,YearBuilt,Link,Seller,Size,SalePrice,lastUpdate


In [174]:
def readAds(webPage):
    d = []
    colNames = ['AdID','Title','Type', 'YearBuilt', 'Link', 'Seller', 'Size','SalePrice', 'lastUpdate']
    data = pd.DataFrame(d, columns= colNames)
    
    for ad in webPage.find_all('div', itemprop='item'): 
        list1 = []

        list1.append(int(ad.find('a', class_='ikona-sh3 utility')['data-id']))

        list1.append(ad.find('span', class_='title').text)

        list1.append(ad.find('span', class_='vrsta').text)

        year_raw = ad.find('span', class_='atribut leto')
        year = int(year_raw.text.split(':')[-1]) if year_raw != None else np.nan
        list1.append(year)

        list1.append(ad.find('a', class_='slika')['href'])

        list1.append(ad.find('div', class_='prodajalec_o')['title'])

        size_raw = ad.find('span', class_='velikost').text
        size = float((size_raw.split(' ')[0]).replace('.', '').replace(',','.')) if size_raw != None else np.nan
        list1.append(size)

        price_raw = ad.find('span', class_='cena').text
        if bool(price_raw) is not False: 
            price = float(((price_raw.split(' ')[0]).replace('.', '')).replace(',', '.')) 
          
        else: 
             price = np.nan# remove dot separation and y decimal sep.
        list1.append(price)

        #date when it gets appended ot the list
        date = datetime.datetime.now()
        list1.append(date)

        dfx = pd.Series(list1,colNames)
        data = data.append(dfx,ignore_index = True)
    return data

In [12]:
# testing if the links are full
for link in tqdm(data1['Link']): print(link)

0it [00:00, ?it/s]


In [173]:

for ad in webPage.find_all('div', itemprop='item'): 
        list1 = []

        list1.append(int(ad.find('a', class_='ikona-sh3 utility')['data-id']))

        list1.append(ad.find('span', class_='title').text)

        list1.append(ad.find('span', class_='vrsta').text)

        year_raw = ad.find('span', class_='atribut leto')
        year = int(year_raw.text.split(':')[-1]) if year_raw != None else np.nan
        list1.append(year)

        list1.append(ad.find('a', class_='slika')['href'])

        list1.append(ad.find('div', class_='prodajalec_o')['title'])

        size_raw = ad.find('span', class_='velikost').text
        size = float((size_raw.split(' ')[0]).replace('.', '').replace(',','.')) if size_raw != None else np.nan
        list1.append(size)

        price_raw = ad.find('span', class_='cena').text
        if bool(price_raw) is not False: 
            price = float(((price_raw.split(' ')[0]).replace('.', '')).replace(',', '.')) 
          
        else: 
             price = np.nan# remove dot separation and y decimal sep.
        list1.append(price)

        # date when it gets appended ot the list
        date = datetime.datetime.now()
        list1.append(date)

        dfx = pd.Series(list1,colNames)
        data = data.append(dfx,ignore_index = True)


,AdID,Title,Type,YearBuilt,Link,Seller,Size,SalePrice,lastUpdate
0,6327522,BEGUNJE NA GORENJSKEM,Počitniški objekt,1989,/oglasi-prodaja/begunje-na-gorenjskem-pocitnis...,ABC nepremičnine d.o.o.,800.0,1.3e+06,2021-03-08 12:54:38.437100
1,6141186,BENEDIKT,Stanovanje,2010,/oglasi-prodaja/benedikt-stanovanje_6141186/,"DUTB, d.d.",52.7,62900,2021-03-08 12:54:38.442087
2,6390803,"BEŽIGRAJSKI DVOR, PLAVA LAGUNA",Stanovanje,1974,/oglasi-prodaja/bezigrajski-dvor-plava-laguna-...,"QUDOS, center nepremičnin d.o.o.",119.0,323000,2021-03-08 12:54:38.450066
3,6327267,BLED,Stanovanje,1984,/oglasi-prodaja/bled-stanovanje_6327267/,Adut nepremičnine d.o.o.,96.1,150000,2021-03-08 12:54:38.457047
4,6235186,BLED,Poslovni prostor,1800,/oglasi-prodaja/bled-poslovni-prostor_6235186/,K3 Kern d.o.o.,548.8,1.1e+06,2021-03-08 12:54:38.462033
...,...,...,...,...,...,...,...,...,...
209,5631844,BRJE,Posest,NaN,/oglasi-prodaja/brje-posest_5631844/,Svet Real d.o.o.,8403.0,2.5,2021-03-08 14:07:46.905193
210,6255750,BRJE PRI KOMNU,Poslovni prostor,1954,/oglasi-prodaja/brje-pri-komnu-poslovni-prosto...,"RE/MAX Ljubljana, Nepremičninska družba d.o.o.",173.0,200000,2021-03-08 14:07:46.909182
211,6041692,BRJE PRI KOMNU,Posest,NaN,/oglasi-prodaja/brje-pri-komnu-posest_6041692/,Renes d.o.o.,46487.0,25000,2021-03-08 14:07:46.914169
212,5719996,"BRJE PRI KOMNU, KOMEN OKOLICA",Hiša,1830,/oglasi-prodaja/brje-pri-komnu-komen-okolica-h...,"Ruj Nepremičnine, Jernej Suša, s.p.",194.0,114900,2021-03-08 14:07:46.919155


In [77]:
d = []
colNames = ['AdID','Title','Type', 'YearBuilt', 'Link', 'Seller', 'Size','SalePrice', 'lastUpdate']
data = pd.DataFrame(d, columns= colNames)

In [168]:
for index, ad in enumerate(webPage.find_all('div', itemprop='item')):
    print({index}, (int(ad.find('a', class_='ikona-sh3 utility')['data-id'])))
    year_raw = ad.find('span', class_='atribut leto')
   # print(year_raw)
    year = int(year_raw.text.split(':')[-1]) if year_raw != None else print(np.nan)
   # print(year)
    
    price_raw = ad.find('span', class_='cena').text
    
    print(bool(price_raw))
    
    if bool(price_raw) is not False: 
        price = float(((price_raw.split(' ')[0]).replace('.', '')).replace(',', '.')) 
        print(price_raw)
        
    else: 
         price = np.nan
   # price_raw_text = price_raw.text if bool(price_raw.text) is not False else np.nan
   # price = float(((price_raw_text.split(' ')[0]).replace('.', '')).replace(',', '.')) 
    print(price)
    print('------------')


{0} 6244607
nan
True
20,00 €/m2
20.0
------------
{1} 6263266
nan
True
42.360,00 €
42360.0
------------
{2} 6244609
nan
True
20,00 €/m2
20.0
------------
{3} 5357631
True
1.000.000,00 €
1000000.0
------------
{4} 6164677
True
38.000,00 €
38000.0
------------
{5} 6368995
True
108.000,00 €
108000.0
------------
{6} 6141403
True
675,00 €/m2
675.0
------------
{7} 6382556
nan
True
220.000,00 €
220000.0
------------
{8} 6389607
False
nan
------------
{9} 6387879
True
140.750,00 €
140750.0
------------
{10} 6324690
True
39.000,00 €
39000.0
------------
{11} 6380075
True
900,00 €/m2
900.0
------------
{12} 6375692
True
220.000,00 €
220000.0
------------
{13} 6169930
nan
True
22.000,00 €
22000.0
------------
{14} 6392535
True
670.000,00 €
670000.0
------------
{15} 6376409
True
541.000,00 €
541000.0
------------
{16} 6352000
True
559.000,00 €
559000.0
------------
{17} 6351997
True
671.000,00 €
671000.0
------------
{18} 6351792
True
671.000,00 €
671000.0
------------
{19} 6182551
True
2.700.0

In [80]:
data2 = readAds(webPage)
data2

,AdID,Title,Type,YearBuilt,Link,Seller,Size,SalePrice,lastUpdate
0,6327522,BEGUNJE NA GORENJSKEM,Počitniški objekt,1989,/oglasi-prodaja/begunje-na-gorenjskem-pocitnis...,ABC nepremičnine d.o.o.,800.00,1300000.0,2021-03-08 12:59:59.949307
1,6141186,BENEDIKT,Stanovanje,2010,/oglasi-prodaja/benedikt-stanovanje_6141186/,"DUTB, d.d.",52.70,62900.0,2021-03-08 12:59:59.957324
2,6390803,"BEŽIGRAJSKI DVOR, PLAVA LAGUNA",Stanovanje,1974,/oglasi-prodaja/bezigrajski-dvor-plava-laguna-...,"QUDOS, center nepremičnin d.o.o.",119.00,323000.0,2021-03-08 12:59:59.966263
3,6327267,BLED,Stanovanje,1984,/oglasi-prodaja/bled-stanovanje_6327267/,Adut nepremičnine d.o.o.,96.10,150000.0,2021-03-08 12:59:59.973249
4,6235186,BLED,Poslovni prostor,1800,/oglasi-prodaja/bled-poslovni-prostor_6235186/,K3 Kern d.o.o.,548.80,1100000.0,2021-03-08 12:59:59.978233
5,6387983,BODEŠČE,Hiša,1800,/oglasi-prodaja/bodesce-hisa_6387983/,Zasebna ponudba,300.00,290000.0,2021-03-08 12:59:59.986209
6,6320190,BREZOVICA PRI LJUBLJANI,Hiša,2007,/oglasi-prodaja/brezovica-pri-ljubljani-hisa_6...,ABC nepremičnine d.o.o.,495.00,399000.0,2021-03-08 12:59:59.991195
7,6272771,"BREZOVICA PRI LJUBLJANI, SREDIŠČE BREZOVICE",Posest,NaN,/oglasi-prodaja/brezovica-pri-ljubljani-sredis...,Dajč & Dajč d.o.o.,1529.00,185.0,2021-03-08 12:59:59.995184
8,6129674,BRITOF,Posest,NaN,/oglasi-prodaja/britof-posest_6129674/,"DUTB, d.d.",21939.00,1950000.0,2021-03-08 13:00:00.000171
9,6389137,BRTONIGLA,Hiša,2021,/oglasi-prodaja/brtonigla-hisa_6389137/,Universum d.o.o.,233.00,380000.0,2021-03-08 13:00:00.005158


In [93]:
# adding df of every page to the main df.
def addDF(mainDF, newDF):
    df = pd.concat([mainDF, newDF],ignore_index=True)
    return df

In [172]:
test1 = addDF(data2, data2)
test1.shape

(120, 9)

In [87]:
# test for generating new URLs
base_url = 'https://www.nepremicnine.net/oglasi-prodaja/'
for i in range(10):
    url = base_url + str(i)+'/'
    print(url)

https://www.nepremicnine.net/oglasi-prodaja/0/
https://www.nepremicnine.net/oglasi-prodaja/1/
https://www.nepremicnine.net/oglasi-prodaja/2/
https://www.nepremicnine.net/oglasi-prodaja/3/
https://www.nepremicnine.net/oglasi-prodaja/4/
https://www.nepremicnine.net/oglasi-prodaja/5/
https://www.nepremicnine.net/oglasi-prodaja/6/
https://www.nepremicnine.net/oglasi-prodaja/7/
https://www.nepremicnine.net/oglasi-prodaja/8/
https://www.nepremicnine.net/oglasi-prodaja/9/


In [186]:
driver = webdriver.Chrome(PATH) 
base_url = 'https://www.nepremicnine.net/oglasi-prodaja/stanovanje/'
base_df = data.iloc[0:0]
df1 = data.iloc[0:0]
start = 0
end = 162
for i in tqdm(range(start+1,end+1)):
    url = base_url + str(i) + '/'
    driver.get(url)
    
   # time.sleep(0.2 + random.randint(1,6)/10)
    html = driver.page_source
    webPage = bs4.BeautifulSoup(html, "lxml")
    
    df1 = readAds(webPage)
    base_df = addDF(base_df,df1)
driver.quit()
base_df.to_csv("./nepremicnine_Stanovanje_" + str(start) + '-'+ str(end) + ".csv")

100%|████████████████████████████████████████████████████████████████████████████████| 162/162 [03:19<00:00,  1.23s/it]


In [187]:
data1 = pd.read_csv('nepremicnine_Stanovanje0-162.csv', index_col = 0,sep = ',')

In [189]:
data1

,AdID,Title,Type,YearBuilt,Link,Seller,Size,SalePrice,lastUpdate
0,6141186,BENEDIKT,Stanovanje,2010.0,/oglasi-prodaja/benedikt-stanovanje_6141186/,"DUTB, d.d.",52.7,62900.0,2021-03-08 16:15:58.652828
1,6390803,"BEŽIGRAJSKI DVOR, PLAVA LAGUNA",Stanovanje,1974.0,/oglasi-prodaja/bezigrajski-dvor-plava-laguna-...,"QUDOS, center nepremičnin d.o.o.",119.0,323000.0,2021-03-08 16:15:58.660773
2,6327267,BLED,Stanovanje,1984.0,/oglasi-prodaja/bled-stanovanje_6327267/,Adut nepremičnine d.o.o.,96.1,150000.0,2021-03-08 16:15:58.666756
3,6389036,BS 3,Stanovanje,1981.0,/oglasi-prodaja/bs-3-stanovanje_6389036/,Studio 68 d.o.o.,73.0,210000.0,2021-03-08 16:15:58.673737
4,6391861,"BTC, BALKON - DVIGALO - NOVA FASADA",Stanovanje,1978.0,/oglasi-prodaja/btc-balkon-dvigalo-nova-fasada...,VIN inženiring d.o.o.,61.2,243700.0,2021-03-08 16:15:58.678724
...,...,...,...,...,...,...,...,...,...
4844,6263207,"ŽELEZNIKI, RACOVNIK",Stanovanje,1952.0,/oglasi-prodaja/zelezniki-racovnik-stanovanje_...,Blok 5 d.o.o.,107.0,64000.0,2021-03-08 16:19:15.631534
4845,6349957,ŽELIMLJE,Stanovanje,2000.0,/oglasi-prodaja/zelimlje-stanovanje_6349957/,"Stoja trade, d.o.o., Ljubljana",124.7,169900.0,2021-03-08 16:19:15.637518
4846,6371534,ŽEPOVCI,Stanovanje,1860.0,/oglasi-prodaja/zepovci-stanovanje_6371534/,Market center Omnis d.o.o.,68.8,49900.0,2021-03-08 16:19:15.642511
4847,6043427,"ŽIRI, CENTER",Stanovanje,2016.0,/oglasi-prodaja/ziri-center-stanovanje_6043427/,Akva d.o.o. PE nepremičnine,100.0,135000.0,2021-03-08 16:19:15.649519
